<a href="https://colab.research.google.com/github/illusoryTwin/InnoML/blob/main/ML_task1_2304(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [95]:
!cp drive/MyDrive/ML/Assignment2/task_1_train_data.json task_1_train_data.json
!cp drive/MyDrive/ML/Assignment2/task_1_verify.json task_1_verify.json

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import sklearn

In [97]:
data = pd.read_json('task_1_train_data.json')
waldo = pd.read_json('task_1_verify.json')

data.head()

,browser,os,locale,user_id,location,sites,time,date
0,Chrome,Debian,ur-PK,116,Russia/Moscow,"[{'site': 'bing.net', 'length': 52}, {'site': ...",04:12:00,2017-06-29
1,Firefox,Windows 8,uk-UA,155,France/Paris,"[{'site': 'yahoo.com', 'length': 46}, {'site':...",03:57:00,2016-03-23
2,Safari,MacOS,fr-FR,39,Japan/Tokyo,"[{'site': 'oracle.com', 'length': 335}]",05:26:00,2016-11-17
3,Chrome,Windows 8,nl-NL,175,Australia/Sydney,"[{'site': 'mail.google.com', 'length': 192}, {...",00:05:00,2016-08-23
4,Firefox,Ubuntu,ro-RO,50,USA/San Francisco,"[{'site': 'mail.google.com', 'length': 266}, {...",22:55:00,2016-07-23


In [ ]:
sites_list = [site_dict['site'] for row in data['sites'] for site_dict in row]

# Count the occurrences of each site
site_counts = pd.Series(sites_list).value_counts()

# Sort the sites based on their occurrences
sorted_sites = site_counts.index.tolist()

# Retrieve the top 30 sites
top_10_sites = sorted_sites[:10]

print(top_10_sites)

['youtube.com', 'toptal.com', 'slack.com', 'lenta.ru', 'vk.com', 'mail.google.com', 'oracle.com', 'wikimedia.org', 'googleapis.com', 'vimeo.com']


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   browser   40000 non-null  object        
 1   os        40000 non-null  object        
 2   locale    40000 non-null  object        
 3   user_id   40000 non-null  int64         
 4   location  40000 non-null  object        
 5   sites     40000 non-null  object        
 6   time      40000 non-null  object        
 7   date      40000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 2.4+ MB


In [98]:
data['target'] = (data.user_id == 0).apply(int)
data['target'].value_counts()

target
0    39800
1      200
Name: count, dtype: int64

In [ ]:
# from datetime import datetime
# from sklearn.preprocessing import OneHotEncoder
# import pandas as pd
# import pytz


# def prepare_dataframe(df: pd.DataFrame):
#     # Convert 'date' column to datetime objects
#     df['date'] = pd.to_datetime(df['date'])

#     # Convert 'time' column to datetime objects
#     df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')  # Assuming the time format is 'hour:minute:second'

#     # Localize 'date' and 'time' columns to UTC timezone
#     df['date'] = df['date'].apply(lambda x: x if x.tzinfo else pytz.utc.localize(x))
#     df['time'] = df['time'].apply(lambda x: x if x.tzinfo else pytz.utc.localize(x))

#     df['day'] = df['date'].apply(lambda x: x.day)
#     df['month'] = df['date'].apply(lambda x: x.month)
#     df['weekday'] = df['date'].apply(lambda x: x.day_of_week)

#     df['browser'] = pd.Categorical(df['browser'])
#     df['os'] = pd.Categorical(df['os'])
#     df['locale'] = pd.Categorical(df['locale'])
#     df['location'] = pd.Categorical(df['location'])

#     df['hour'] = df['time'].apply(lambda x: x.hour)
#     df['minute'] = df['time'].apply(lambda x: x.minute)
#     df['sites_lengths'] = df['sites'].apply(lambda x: len(x))

#     df = cyclic_encode(df, 'day', 365)
#     df = cyclic_encode(df, 'month', 12)
#     df = cyclic_encode(df, 'weekday', 7)
#     df = cyclic_encode(df, 'hour', 24)
#     df = cyclic_encode(df, 'minute', 60)

#     return df.drop(columns=['sites', 'time', 'date'])

# # Assuming cyclic_encode is a function to perform cyclical encoding
# def cyclic_encode(df, column, max_value):
#     df[column + '_sin'] = np.sin(2 * np.pi * df[column] / max_value)
#     df[column + '_cos'] = np.cos(2 * np.pi * df[column] / max_value)
#     return df

# # Example usage:
# y = data['target']
# X = data.drop(columns=['target', 'user_id'])
# train_df = prepare_dataframe(X)
# train_df.head()


In [99]:
# encode cyclical features
def cyclic_encode(df, col, period):
    df[f'sin_{col}'] = np.sin(2 * np.pi * df[col].astype(int) / period)
    df[f'cos_{col}'] = np.cos(2 * np.pi * df[col].astype(int) / period)
    return df.drop(columns=[col])

In [100]:
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import pytz




def prepare_dataframe(df: pd.DataFrame):

    df['day'] = df['date'].apply(lambda x: x.day)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['weekday'] = df['date'].apply(lambda x: x.day_of_week)

    df['browser'] = pd.Categorical(df['browser'])
    df['os'] = pd.Categorical(df['os'])
    df['locale'] = pd.Categorical(df['locale'])
    df['location'] = pd.Categorical(df['location'])

    df['hour'] = df['time'].apply(lambda x: int(str(x).split(':')[0]))
    df['minute'] = df['time'].apply(lambda x: int(str(x).split(':')[1]))
    # df['sites_lengths'] = df['sites'].apply(lambda x: len(x))
    # for site in top_10_sites:
    #   df[site] = df['sites'].apply(lambda x: 1 if any(site_dict['site'] == site for site_dict in x) else 0)

    # for site in top_5_sites:
    #         df[site] = 0  # Initialize time spent on each site to 0

    # # Step 6: Iterate over each row and calculate the time spent on each site
    # for index, row in df.iterrows():
    #     sites = row['sites']
    #     for site_dict in sites:
    #         site = site_dict['site']
    #         if site in top_5_sites:
    #             df.at[index, site] += site_dict['length']



    df = cyclic_encode(df, 'day', 365)
    df = cyclic_encode(df, 'month', 12)
    df = cyclic_encode(df, 'weekday', 7)
    df = cyclic_encode(df, 'hour', 24)
    df = cyclic_encode(df, 'minute', 60)

    return df.drop(columns=['sites', 'time', 'date'])

y = data['target']
X = data.drop(columns=['target', 'user_id'])
train_df = prepare_dataframe(X)

train_df.head()

,browser,os,locale,location,sin_day,cos_day,sin_month,cos_month,sin_weekday,cos_weekday,sin_hour,cos_hour,sin_minute,cos_minute
0,Chrome,Debian,ur-PK,Russia/Moscow,0.478734,0.877960,1.224647e-16,-1.000000e+00,0.433884,-0.900969,0.866025,0.500000,0.951057,0.309017
1,Firefox,Windows 8,uk-UA,France/Paris,0.385663,0.922640,1.000000e+00,6.123234e-17,0.974928,-0.222521,0.707107,0.707107,-0.309017,0.951057
2,Safari,MacOS,fr-FR,Japan/Tokyo,0.288482,0.957485,-5.000000e-01,8.660254e-01,0.433884,-0.900969,0.965926,0.258819,0.406737,-0.913545
3,Chrome,Windows 8,nl-NL,Australia/Sydney,0.385663,0.922640,-8.660254e-01,-5.000000e-01,0.781831,0.623490,0.000000,1.000000,0.500000,0.866025
4,Firefox,Ubuntu,ro-RO,USA/San Francisco,0.385663,0.922640,-5.000000e-01,-8.660254e-01,-0.974928,-0.222521,-0.500000,0.866025,-0.500000,0.866025


In [101]:
ohe = OneHotEncoder(sparse_output=False)
ohe_data = ohe.fit_transform(train_df[['browser','os','locale','location']])
ohe_data.shape

(40000, 56)

In [102]:
train_data = np.concatenate((ohe_data, train_df.drop(columns=['browser','os','locale','location']).values), axis=1)

train_data.shape

(40000, 66)

In [103]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# pca = PCA(n_components=0.99)
# train_data = pca.fit_transform(train_data)
# train_data.shape
# scale = StandardScaler()
# train_data = scale.fit_transform(train_data)
# train_data.shape

In [104]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train_data, y, shuffle=True, stratify=y, test_size=0.2, random_state=42)

In [105]:
# from sklearn.linear_model import LogisticRegression
# cls = LogisticRegression(max_iter=1000)

# cls.fit(x_train, y_train)

# preds = cls.predict(x_val)
# print(sklearn.metrics.classification_report(y_val, preds))

In [106]:
class ANN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, output_dim)
        )

    def forward(self, x):
        return self.model(x)

In [107]:
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch_x_train = torch.tensor(x_train, dtype=torch.float32)
torch_x_val = torch.tensor(x_val, dtype=torch.float32)

torch_y_train = torch.tensor(y_train.values, dtype=torch.long)
torch_y_val = torch.tensor(y_val.values, dtype=torch.long)

train_dataset = torch.utils.data.TensorDataset(torch_x_train, torch_y_train)
val_dataset = torch.utils.data.TensorDataset(torch_x_val, torch_y_val)

# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2048)


In [108]:
from tqdm import tqdm
def train(model, loss_fn, optim, train_dataloader, scheduler=None):
    model.train()
    losses = []
    for inp, target in train_dataloader:
        inp = inp.to(device)
        inp.requires_grad = True
        target = target.to(device)
        optim.zero_grad()

        preds = model(inp)
        loss = loss_fn(preds, target)
        loss.backward()
        optim.step()
        losses.append(loss.item())
    return np.mean(losses)

def validation(model, val_dataloader, score_fn):
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inp, target in val_dataloader:
            inp = inp.to(device)
            target = target.to(device)
            preds = model(inp).cpu().detach().numpy().argmax(axis=1)
            y_pred.append(preds)
            y_true.append(target.cpu().detach().numpy())
    y_pred = np.hstack(y_pred)
    y_true = np.hstack(y_true)

    return score_fn(y_pred, y_true)



model = ANN(x_train.shape[-1], 2)
model.to(device)
weight=torch.tensor([0.1, 0.9], dtype=torch.float32, device=device)
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=.9)

epochs = 50
best_model_path = 'best.pth'
best_score = 0
patience = 20
pbar = tqdm(range(epochs))
for e in pbar:
    # print(f"{e + 1}/{epochs} Train loss:", train(model, loss_fn, optim, train_dataloader))
    train(model, loss_fn, optim, train_dataloader, scheduler)
    scheduler.step()
    val_score = validation(model, val_dataloader, sklearn.metrics.f1_score)
    if val_score > best_score:
        torch.save(model.state_dict(), best_model_path)
        best_score = val_score
    else:
        patience -= 1
    if patience == 0:
        break
    torch.save(model.state_dict(), best_model_path)

    pbar.set_postfix({'val_score': val_score, 'best_score': best_score})
torch.save(model.state_dict(), best_model_path)


 52%|█████▏    | 26/50 [01:03<00:58,  2.43s/it, val_score=0.644, best_score=0.645]


In [109]:
best_model = ANN(x_train.shape[-1], 2)
best_model.to(device)
best_model.load_state_dict(torch.load(best_model_path))
best_model.eval()

ANN(
  (model): Sequential(
    (0): Linear(in_features=66, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=16, bias=True)
    (6): ReLU()
    (7): Linear(in_features=16, out_features=2, bias=True)
  )
)

In [110]:
best_model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for inp, target in val_dataloader:
        inp = inp.to(device)
        target = target.to(device)
        preds = best_model(inp).cpu().detach().numpy().argmax(axis=1)
        y_pred.append(preds)
        y_true.append(target.cpu().detach().numpy())
y_pred = np.hstack(y_pred)
y_true = np.hstack(y_true)

print(sklearn.metrics.classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9972    0.9997    0.9985      7960
           1     0.9000    0.4500    0.6000        40

    accuracy                         0.9970      8000
   macro avg     0.9486    0.7249    0.7992      8000
weighted avg     0.9968    0.9970    0.9965      8000



In [111]:
def validation(model, val_dataloader, score_fn):
    model.eval()
    y_pred = []
    with torch.no_grad():
        for inp in val_dataloader:
            inp = inp[0]
            inp = inp.to(device)
            preds = model(inp).cpu().detach().numpy().argmax(axis=1)
            y_pred.append(preds)
    y_pred = np.hstack(y_pred)

    return y_pred

In [112]:
verify_df = prepare_dataframe(waldo)
ohe_data_verify = ohe.fit_transform(verify_df[['browser','os','locale','location']])
verify_data = np.concatenate((ohe_data_verify, verify_df.drop(columns=['browser','os','locale','location']).values), axis=1)



torch_verify = torch.tensor(verify_data, dtype=torch.float32)
verify_dataset = torch.utils.data.TensorDataset(torch_verify)
verify_dataloader = torch.utils.data.DataLoader(verify_dataset, batch_size=32)

In [113]:
test_labels = validation(best_model, verify_dataloader, best_score)

In [115]:
len([x for x in test_labels if x == 1])

115

In [74]:
verify_df = prepare_dataframe(waldo)
ohe_data_verify = ohe.fit_transform(verify_df[['browser','os','locale','location']])
verify_data = np.concatenate((ohe_data_verify, verify_df.drop(columns=['browser','os','locale','location']).values), axis=1)



torch_verify = torch.tensor(verify_data, dtype=torch.float32)
verify_dataset = torch.utils.data.TensorDataset(torch_verify)
verify_dataloader = torch.utils.data.DataLoader(verify_dataset, batch_size=32)



best_model.eval()
verify_data_tensor = torch.tensor(verify_data).float().to(device)
verify_output = best_model(verify_data_tensor)
verify_predictions = (torch.sigmoid(verify_output) > best_score).int().cpu().numpy()

count_waldo = 0
for row_idx, is_waldo in enumerate(verify_predictions):
  print(f"{row_idx}: {is_waldo}")

#   count_waldo += is_waldo


# print(count_waldo)

Streaming output truncated to the last 5000 lines.
35000: [1 0]
35001: [1 0]
35002: [1 0]
35003: [1 0]
35004: [1 0]
35005: [1 0]
35006: [1 0]
35007: [1 0]
35008: [1 0]
35009: [1 0]
35010: [1 0]
35011: [1 0]
35012: [1 0]
35013: [1 0]
35014: [1 0]
35015: [1 0]
35016: [1 0]
35017: [1 0]
35018: [1 0]
35019: [1 0]
35020: [1 0]
35021: [1 0]
35022: [1 0]
35023: [1 0]
35024: [1 0]
35025: [1 0]
35026: [1 0]
35027: [1 0]
35028: [1 0]
35029: [1 0]
35030: [1 0]
35031: [1 0]
35032: [1 0]
35033: [1 0]
35034: [1 0]
35035: [1 0]
35036: [1 0]
35037: [1 0]
35038: [1 0]
35039: [1 0]
35040: [1 0]
35041: [1 0]
35042: [1 0]
35043: [1 0]
35044: [1 0]
35045: [1 0]
35046: [1 0]
35047: [1 0]
35048: [1 0]
35049: [1 0]
35050: [1 0]
35051: [1 0]
35052: [1 0]
35053: [1 0]
35054: [1 0]
35055: [1 0]
35056: [1 0]
35057: [1 0]
35058: [1 0]
35059: [1 0]
35060: [1 0]
35061: [1 0]
35062: [1 0]
35063: [1 0]
35064: [1 0]
35065: [0 1]
35066: [1 0]
35067: [1 0]
35068: [1 0]
35069: [1 0]
35070: [1 0]
35071: [1 0]
35072: [1 0]


In [76]:
len(verify_dataloader)

1250

In [48]:
len(verify_predictions)

80000

In [67]:
verify_output[0:20]

tensor([[ 138.7930, -142.3548],
        [  78.9973,  -79.4857],
        [  75.9794,  -77.6970],
        [  98.4456,  -99.2093],
        [  68.8462,  -67.3368],
        [ 129.3214, -128.9766],
        [ 138.0761, -137.7959],
        [  82.0805,  -85.8250],
        [  94.1002,  -93.9608],
        [ 118.4956, -115.4361],
        [  99.5660,  -95.9782],
        [  72.7316,  -73.5415],
        [ 127.9796, -125.4843],
        [  78.2888,  -77.9287],
        [  68.7454,  -71.0671],
        [ 104.7159, -102.8248],
        [ 153.3543, -148.8195],
        [  80.9647,  -81.3927],
        [ 122.7335, -121.7094],
        [ 133.2167, -130.5446]], device='cuda:0', grad_fn=<SliceBackward0>)